## Imports

In [ ]:
import os
import gc
import re
import cv2
import math
import time
import keras
import numpy
import pandas
import random
import tensorflow
import seaborn as sns
from numba import cuda
from PIL import Image
import keras.backend as K
import tensorflow as tf
from typing import Tuple, List
from pandas import DataFrame
import matplotlib.pyplot as plt
from tensorflow.keras.layers import GlobalAveragePooling2D
from keras.optimizers import RMSprop, SGD, Adam, Adamax
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50V2, ResNet152V2, ResNet50, MobileNetV2
from keras.models import Model, Sequential, load_model, Input
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.applications.mobilenet import preprocess_input
from keras.layers.convolutional import Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import load_img, img_to_array

## Setup

In [ ]:
train_csv = pandas.read_csv("../input/plant-pathology-2021-fgvc8/train.csv")
# one_hot_train_csv = pandas.read_csv("../input/plant-pathology-models/one_hot_encoded_train.csv")
# train_images_dir = "../input/plant-pathology-2021-resized/plant_pathology_2021_resized/train_images/620x403"
# train_images_dir = "../input/plant-pathology-2021-resized/512x512_augmented/512x512_augmented"
# train_images_dir = "../input/plant-pathology-2021-resized/512x512_augmented_4/512x512_augmented_4"
train_images_dir = "../input/plant-pathology-2021-resized/512x512_augmented_combined/512x512_augmented_combined"

# train_images_dir = "/kaggle/input/plant-pathology-2021-fgvc8/train_images"
test_images_dir = "../input/plant-pathology-2021-fgvc8/test_images"
ws_splitter = re.compile("\\s+")
CLASSES = [
    'healthy', 
    'scab frog_eye_leaf_spot complex', 
    'scab', 
    'complex', 
    'rust', 
    'frog_eye_leaf_spot', 
    'powdery_mildew', 
    'scab frog_eye_leaf_spot', 
    'frog_eye_leaf_spot complex', 
    'rust frog_eye_leaf_spot', 
    'powdery_mildew complex', 
    'rust complex'
]


DUPLICATE_IMAGES = [
"c5aff2e545d0a129.jpg","c9d63696629107ed.jpg","ae5074cfc9183763.jpg","d64460a3b19f1cfc.jpg",
"9ada65c5b0cc9a3c.jpg","94c83b2d5fad6924.jpg","ba99260ef46b18cb.jpg","a3bb604d8a55995d.jpg",
"949b6595a5b8ca3c.jpg","dea33c5c0347aa4d.jpg","e51690d8ad307dcb.jpg","8c0731fff680c51e.jpg",
"e83f715e56462a31.jpg","9fc321ce60a69f0b.jpg","bd8088cdad5f65e0.jpg","cdf892b5b4865859.jpg",
"f2c798246a0a9bdd.jpg","f7d233d11aa8496c.jpg","c1753cad2f31d492.jpg","f3dc17a2b20d48ec.jpg",
"aa0e969518e15d3f.jpg","b5ec52491e3349ce.jpg","ec83c7c04a1e9db9.jpg","e5f27342ce1cb9a0.jpg",
"9fad869f21b5b240.jpg","d1f01e8d8a73274d.jpg","d1dec0973c2cf4a1.jpg","dbee22785d9285c4.jpg",
"8f5263c61a8cdbe2.jpg","965295a64a97859f.jpg","f4cba4a7294d6585.jpg","e9e8ad35c5871d06.jpg",
"90f99ee1c1132bb6.jpg","b7b0edd58a858217.jpg","e7ecc583d571d202.jpg","848d9f9e82d18e5e.jpg",
"d3945c098edc9dd1.jpg","eec64e8cf1c8d0c6.jpg","df90901ccb664de6.jpg","d0c766bc38e1978c.jpg",
"86f963a16e70345e.jpg","f90738a0d2f2c1db.jpg","82ba3e933947b097.jpg","c1a56069de75b899.jpg",
"94b42695f24dad36.jpg","d2266ac99461e9db.jpg","90a5341f227ce5f9.jpg","bb1891fa8c61e663.jpg",
"a7e613159fb22859.jpg","85897ea7f064b26a.jpg","ca782c131f5ce0fc.jpg","d7ec32d72c84dc18.jpg",
"a9b2e4b09297bc2b.jpg","d2d9c2bca7a72458.jpg",
"augmented_c5aff2e545d0a129.jpg","augmented_c9d63696629107ed.jpg","augmented_ae5074cfc9183763.jpg",
"augmented_d64460a3b19f1cfc.jpg","augmented_9ada65c5b0cc9a3c.jpg","augmented_94c83b2d5fad6924.jpg",
"augmented_ba99260ef46b18cb.jpg","augmented_a3bb604d8a55995d.jpg","augmented_949b6595a5b8ca3c.jpg",
"augmented_dea33c5c0347aa4d.jpg","augmented_e51690d8ad307dcb.jpg","augmented_8c0731fff680c51e.jpg",
"augmented_e83f715e56462a31.jpg","augmented_9fc321ce60a69f0b.jpg","augmented_bd8088cdad5f65e0.jpg",
"augmented_cdf892b5b4865859.jpg","augmented_f2c798246a0a9bdd.jpg","augmented_f7d233d11aa8496c.jpg",
"augmented_c1753cad2f31d492.jpg","augmented_f3dc17a2b20d48ec.jpg","augmented_aa0e969518e15d3f.jpg",
"augmented_b5ec52491e3349ce.jpg","augmented_ec83c7c04a1e9db9.jpg","augmented_e5f27342ce1cb9a0.jpg",
"augmented_9fad869f21b5b240.jpg","augmented_d1f01e8d8a73274d.jpg","augmented_d1dec0973c2cf4a1.jpg",
"augmented_dbee22785d9285c4.jpg","augmented_8f5263c61a8cdbe2.jpg","augmented_965295a64a97859f.jpg",
"augmented_f4cba4a7294d6585.jpg","augmented_e9e8ad35c5871d06.jpg","augmented_90f99ee1c1132bb6.jpg",
"augmented_b7b0edd58a858217.jpg","augmented_e7ecc583d571d202.jpg","augmented_848d9f9e82d18e5e.jpg",
"augmented_d3945c098edc9dd1.jpg","augmented_eec64e8cf1c8d0c6.jpg","augmented_df90901ccb664de6.jpg",
"augmented_d0c766bc38e1978c.jpg","augmented_86f963a16e70345e.jpg","augmented_f90738a0d2f2c1db.jpg",
"augmented_82ba3e933947b097.jpg","augmented_c1a56069de75b899.jpg","augmented_94b42695f24dad36.jpg",
"augmented_d2266ac99461e9db.jpg","augmented_90a5341f227ce5f9.jpg","augmented_bb1891fa8c61e663.jpg",
"augmented_a7e613159fb22859.jpg","augmented_85897ea7f064b26a.jpg","augmented_ca782c131f5ce0fc.jpg",
"augmented_d7ec32d72c84dc18.jpg","augmented_a9b2e4b09297bc2b.jpg","augmented_d2d9c2bca7a72458.jpg",
"augmented2_c5aff2e545d0a129.jpg","augmented2_c9d63696629107ed.jpg","augmented2_ae5074cfc9183763.jpg",
"augmented2_d64460a3b19f1cfc.jpg","augmented2_9ada65c5b0cc9a3c.jpg","augmented2_94c83b2d5fad6924.jpg",
"augmented2_ba99260ef46b18cb.jpg","augmented2_a3bb604d8a55995d.jpg","augmented2_949b6595a5b8ca3c.jpg",
"augmented2_dea33c5c0347aa4d.jpg","augmented2_e51690d8ad307dcb.jpg","augmented2_8c0731fff680c51e.jpg",
"augmented2_e83f715e56462a31.jpg","augmented2_9fc321ce60a69f0b.jpg","augmented2_bd8088cdad5f65e0.jpg",
"augmented2_cdf892b5b4865859.jpg","augmented2_f2c798246a0a9bdd.jpg","augmented2_f7d233d11aa8496c.jpg",
"augmented2_c1753cad2f31d492.jpg","augmented2_f3dc17a2b20d48ec.jpg","augmented2_aa0e969518e15d3f.jpg",
"augmented2_b5ec52491e3349ce.jpg","augmented2_ec83c7c04a1e9db9.jpg","augmented2_e5f27342ce1cb9a0.jpg",
"augmented2_9fad869f21b5b240.jpg","augmented2_d1f01e8d8a73274d.jpg","augmented2_d1dec0973c2cf4a1.jpg",
"augmented2_dbee22785d9285c4.jpg","augmented2_8f5263c61a8cdbe2.jpg","augmented2_965295a64a97859f.jpg",
"augmented2_f4cba4a7294d6585.jpg","augmented2_e9e8ad35c5871d06.jpg","augmented2_90f99ee1c1132bb6.jpg",
"augmented2_b7b0edd58a858217.jpg","augmented2_e7ecc583d571d202.jpg","augmented2_848d9f9e82d18e5e.jpg",
"augmented2_d3945c098edc9dd1.jpg","augmented2_eec64e8cf1c8d0c6.jpg","augmented2_df90901ccb664de6.jpg",
"augmented2_d0c766bc38e1978c.jpg","augmented2_86f963a16e70345e.jpg","augmented2_f90738a0d2f2c1db.jpg",
"augmented2_82ba3e933947b097.jpg","augmented2_c1a56069de75b899.jpg","augmented2_94b42695f24dad36.jpg",
"augmented2_d2266ac99461e9db.jpg","augmented2_90a5341f227ce5f9.jpg","augmented2_bb1891fa8c61e663.jpg",
"augmented2_a7e613159fb22859.jpg","augmented2_85897ea7f064b26a.jpg","augmented2_ca782c131f5ce0fc.jpg",
"augmented2_d7ec32d72c84dc18.jpg","augmented2_a9b2e4b09297bc2b.jpg","augmented2_d2d9c2bca7a72458.jpg",
".DS_Store"
]


## Data Visualization

In [ ]:
def show_label_frequency():
    frequency = dict()
    for i in range(0, len(train_csv)):
        label = train_csv.iloc[i]["labels"]
        if label in frequency:
            frequency[label] = frequency[label] + 1
        else:
            frequency[label] = 1
    plt.figure(figsize=(15,10))
    plt.bar(range(len(frequency)), list(frequency.values()), align='center')
    plt.xticks(range(len(frequency)), list(frequency.keys()))
    plt.xticks(rotation=90)
    plt.show()

def show_image_sizes():
    train_images = os.listdir(train_images_dir)
    for img_name in train_images:
        img = Image.open("{}/{}".format(train_images_dir, img_name))
        print(img.size)

show_label_frequency()
# train_csv.head()

## Data Generation

In [ ]:
class PlantDataGenerator:
    start: int
    end: int
    batch_size: int
    input_shape: Tuple[int, int, int]
    length = 0
    images = []
    
    def __init__(self, start: int, end: int, batch_size: int, input_shape=(940, 611, 3)):
        self.start = start
        self.end = end
        self.batch_size = batch_size
        self.input_shape = input_shape
        train_images = os.listdir(train_images_dir)
        images = train_images[self.start:self.end]
        for img_name in DUPLICATE_IMAGES:
            if img_name in images:
                images.remove(img_name)
                
        random.shuffle(images)
        self.images = images
        self.length = len(train_images[self.start:self.end])
    
    def generate(self):
        batch_start = 0
        batch_end = self.batch_size
        images = self.images
        while True:
            batch_images = images[batch_start:batch_end]
            x_inputs = []
            y_outputs = []
            # start = time.time()
            for image_name in batch_images:
                img_id = image_name
                if "augmented_" in img_id:
                    img_id = img_id.replace("augmented_", "")
                elif "augmented2_" in img_id:
                    img_id = img_id.replace("augmented2_", "")
                img_path = "{}/{}".format(train_images_dir, image_name)
                label = train_csv.loc[train_csv["image"] == img_id].iloc[0]["labels"]
                img = Image.open(img_path)
#                 img = img.resize(self.input_shape[0:2])
                img = numpy.array(img)
                y = []
                for class_name in CLASSES:
                    if class_name == label:
                        y.append(1.0)
                    else:
                        y.append(0.0)
                y = numpy.array(y)
                y_outputs.append(y)
                x_inputs.append(img)
#                 print("{}: {}: {}".format(image_name, y, labels))
            # elapsed = time.time() - start
            # print("Loaded {} batch in {} seconds".format(self.batch_size, elapsed))
            x_inputs = numpy.array(x_inputs)
            y_outputs = numpy.array(y_outputs)
            yield x_inputs, y_outputs
            batch_start += self.batch_size
            batch_end += self.batch_size
            if batch_end > len(images) and batch_start >= len(images):
                batch_start = 0
                batch_end = self.batch_size
                random.shuffle(self.images)
            elif batch_end >= len(images):
                batch_end = len(images) - 1

## Submission

In [ ]:
def submit(model: Model, img_dir: str, output_dir: str, input_shape: Tuple[int, int, int], threshold=0.5):
    images = os.listdir(img_dir)
    print("Starting submission!")
    output = open("{}/submission.csv".format(output_dir), "w")
    output.write("image,labels")
    count = 0
    total_start = time.time()
    for img_name in images:
        start = time.time()
        img_path = "{}/{}".format(img_dir, img_name)
        img = Image.open(img_path)
        img = img.resize(input_shape[0:2])
        img = numpy.array(img)
        x = img.reshape((1,) + input_shape)
        prediction = CLASSES[numpy.argmax(model.predict(x)[0])]
        output.write("\n{},{}".format(img_name, prediction))
        elapsed = time.time() - start
        count += 1
        if count % 500 == 0:
            try:
                minutes = (time.time() - total_start) / 60
                percent_complete = (count / len(images)) * 100.0
                print("[{}/{}]: Made {} predictions in {:.2f} minutes. {:.2f}% complete".format(count, len(images), count, minutes, percent_complete))
            except Exception as e:
                pass
    total_elapsed = (time.time() - total_start) / 60.0
    print("\n\nCompleted {} predictions in {:.2f} minutes!".format(count, total_elapsed))
    print("_________________________________________________________")
    output.close()

## Model

In [ ]:
def plant_model13(input_shape) -> Sequential:
    resnet = ResNet50(weights="imagenet", include_top=False, input_shape=input_shape)
    resnet.trainable = True
    model = Sequential(name="plant_model13")
    model.add(resnet)
    model.add(Flatten())
    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.5))
    model.add(BatchNormalization())
    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.5))
    model.add(BatchNormalization())
    model.add(Dense(512, activation="relu"))
    model.add(Dropout(0.5))
    model.add(BatchNormalization())
    model.add(Dense(len(CLASSES), activation="softmax"))
    model.compile(optimizer=SGD(lr=5e-4), loss="categorical_crossentropy", metrics=["accuracy"])
    model.summary()
    return model

## Validation

In [ ]:
def format_label_string(y, threshold):
    label = ""
    for i in range(0, len(y)):
        if y[i] > threshold:
            if i == 0:
                label = "{} ".format(CLASSES[i])
            else:
                label = "{}{} ".format(label, CLASSES[i])
    if label == "":
        label = "healthy"
    label = label.strip()
    return label

def validate(model: Model, start: int, end: int, input_shape: Tuple[int, int, int], batch_size=8, threshold=0.8):
    data_gen = PlantDataGenerator(start, end, batch_size, input_shape=input_shape)
    data_set = data_gen.generate()
    batches = int(numpy.ceil(data_gen.length / batch_size)) - 1
    correct = 0
    total = 0
    
    
    predictions_out = open("predictions.csv", "w")
    predictions_out.write("Correct,Prediction,Answer")
    for class_name in CLASSES:
        predictions_out.write(",{}".format(class_name))
    print("Validating {}".format(model.name))
    for k in range(0, batches):
        batch_x, batch_y = data_set.__next__()
        for i in range(0, len(batch_x)):
            x = batch_x[i].reshape((1,) + input_shape)
            y = CLASSES[numpy.argmax(batch_y[i])]
            probabilities = model.predict(x)[0]
            prediction = CLASSES[numpy.argmax(probabilities)]
            predictions_out.write("\n{},{},{}".format(
                y == prediction, 
                prediction, y
            ))
            for p in probabilities:
                predictions_out.write(",{}".format(p))
            if y == prediction:
                correct += 1
            if total % 100 == 0:
                print("[{}/{}]: A: {}, P: {}".format(total, batches * batch_size, y, prediction))
            total += 1
    predictions_out.close()
    avg_acc = correct / total
    print("\n\Correctly predicted {}/{} with:\n\tAvg. Accuracy: {}".format(correct, total, avg_acc))
    print("\n____________________________________________________")

## Training

In [ ]:
def train(
    model: Model, 
    epochs: int, 
    start: int, 
    end: int, 
    input_shape: Tuple[int, int, int], 
    batch_size=6, 
    validation_split=0.1, 
    use_augmentation=False):
    start_time = time.time()
    if use_augmentation:
        data_set = load_augmented_data(input_shape, validation_split, batch_size)
        history = model.fit(data_set, epochs=epochs, batch_size=batch_size)
    else:
        data_gen = PlantDataGenerator(start, end, batch_size, input_shape=input_shape)
        data_set = data_gen.generate()
        steps_per_epoch = int(numpy.ceil(data_gen.length / batch_size)) - 1
        history = model.fit(
            data_set, 
            epochs=epochs, 
            batch_size=batch_size, 
            steps_per_epoch=steps_per_epoch
        )
    model.save("{}.keras".format(model.name))
    elapsed = time.time() - start_time
    if "accuracy" in history.history:
        avg_accuracy = numpy.average(history.history["accuracy"])
        avg_loss = numpy.average(history.history["loss"])
        print(
            "\n\nCompleted {} epoch in {} seconds with:\n\tAvg. Accuracy: {}%\n\tAvg. Loss: {}".format(
                epochs, 
                elapsed, 
                avg_accuracy, 
                avg_loss
            )
        )
    else:
        print("\n\nCompleted {} epoch(s) in {} seconds:".format(epochs, elapsed))
        for key in history.history:
            if "accuracy" in key or "loss" in key:
                print("   {}: {}%".format(key, numpy.average(history.history[key])))
        print("\n___________________________________________")
    return history

## Runtime

In [ ]:
# input_shape = (940, 611, 3)
input_shape = (512, 512, 3)
dstart = 6000
dend = None
validation_split = 0.1
batch_size = 16
# model = plant_model13(input_shape)
model = load_model("/kaggle/input/d/tylerswann/plant-pathology-models/plant_model9_14.keras")
# history = train(model, epochs=10, start=dstart, end=dend, input_shape=input_shape, batch_size=batch_size, validation_split=validation_split, use_augmentation=False)
# validate(model, 0, dstart, input_shape, threshold=0.5)
submit(model, test_images_dir, "/kaggle/working", input_shape)

## Metrics

In [ ]:
try:
    if history is not None:
        fig, axes = plt.subplots(1, 2, figsize=(30,10))
        axes[0].plot(history.history["accuracy"])
        axes[0].set_title('Model Accuracy')
        axes[0].set_ylabel('Accuracy')
        axes[0].set_xlabel('Epoch')
        axes[1].plot(history.history["loss"])
        axes[1].set_title('Model Loss')
        axes[1].set_ylabel('Loss')
        axes[1].set_xlabel('Epoch')
        plt.show()
except Exception as e:
    pass

## Deallocate Memory

In [ ]:
cuda.select_device(0)
cuda.close()
del model
keras.backend.clear_session()

<a href="predictions.csv">predictions.csv</a>

<a href="disease_classification.keras">disease_classification.keras</a>